In [1]:
import numpy as np
from sklearn.cluster import SpectralClustering,KMeans
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef,accuracy_score,precision_score,recall_score,confusion_matrix
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import os

In [2]:
# HL: transfer from dataden "/umms-kinfai/duolin/ying/reditools2_candidates/"
hash_candidat={}
hash_candidat['AFG-H1_directRNA']='H1-AFG.candidate_sites.tab'
hash_candidat['AFG-H9_directRNA']='H9-AFG.candidate_sites.tab'
hash_candidat["PGC-H1_directRNA"]='H1-PGC.candidate_sites.tab'
hash_candidat["DE-H1_directRNA"]='H1-DE.candidate_sites.tab'
hash_candidat["DE-H9_directRNA"]='H9-DE.candidate_sites.tab'
hash_candidat["GM12878_directRNA"]='GM12878.candidate_sites.tab'
hash_candidat["H1-hESC_directRNA"]='H1-hESC.candidate_sites.tab'
hash_candidat["H9-hESC_directRNA"]='H9-hESC.candidate_sites.tab'
hash_candidat['HEK293T_DKO_directRNA']='HEK293T_WT.candidate_sites.tab'
hash_candidat["HEK293T_WT_directRNA"]='HEK293T_WT.candidate_sites.tab'
hash_candidat["HCT116_WT"]='HCT116.candidate_sites.tab'
hash_candidat["HEK293T_WT"]='HEK293T_WT.candidate_sites.tab'


In [3]:
long_reads_min_coverage = coverage_cutoff = 5

In [4]:
covfilename=5
coverage_cutoff= 5
datatype = 'HEK293T_WT'
balance_neg=True
inputfolder="/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure2e/ReDD_results/release_general_model"

filename1=inputfolder + '/'+"HEK_WT_pass_onlycandidate_cov"+str(covfilename)+"_ratio0_modcov0.sitelev.bed" #background must use no filter except coverage
filename2=inputfolder + '/'+"HEK_WT_pass_noncandidate_cov"+str(covfilename)+"_ratio0_modcov0.sitelev.bed"

candidatefile="/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure2a/reditools2_candidates/"+hash_candidat[datatype]
rediportal = "/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure3a/REDIportal_hg38.txt"

using_mean_prediction_score=True
if using_mean_prediction_score:
   score_index=-1
else:
   score_index=-2

true_candidate_ratios={}
shortreadcoverage={}
input = open(candidatefile,'r')
for line in input:
    chr_ = line.split()[0]
    pos_ = line.split()[1]
    true_candidate_ratios[chr_+"-"+pos_]=float(line.split("\t")[3])
    shortreadcoverage[chr_+"-"+pos_]=float(line.split("\t")[-1])

REDIportal_sites={}
input = open(rediportal,'r')
for line in input:
    chr_ = line.split()[0]
    pos_ = line.split()[1]
    REDIportal_sites[chr_+"-"+pos_]=1


true_value_all=[]
predict_value_all=[]
num_candidate=0
index_neg=[]
input=open(filename1)
for line in input:
    readcoverage=int(line.split()[3])
    if readcoverage<coverage_cutoff:
         continue

    chr_ = line.split()[0]
    pos_ = line.split()[1]
    if len(line.split("\t")) == 6:
        predictscore=float(line.split("\t")[score_index]) #last score is mean prediciton score
    else:
        predictscore=float(line.split("\t")[-1]) #for old result there is no mean prediction scores

    if chr_+"-"+pos_ in true_candidate_ratios:
         true_value_all.append(true_candidate_ratios[chr_+"-"+pos_])
    else:
         if chr_+"-"+pos_  in REDIportal_sites:
              continue  #remove all sites in REDIportal_sites from neg set

         true_value_all.append(0)
         index_neg.append(len(true_value_all)-1)

    predict_value_all.append(predictscore)
    num_candidate+=1

input=open(filename2)
num_noncandidate=0
for line in input:
    readcoverage=int(line.split()[3])
    if readcoverage<coverage_cutoff:
         continue

    chr_ = line.split()[0]
    pos_ = line.split()[1]
    if len(line.split("\t")) == 6:
        predictscore=float(line.split("\t")[score_index])
    else:
        predictscore=float(line.split("\t")[-1])

    if chr_+"-"+pos_ in true_candidate_ratios:
         true_value_all.append(true_candidate_ratios[chr_+"-"+pos_])
    else:
         if chr_+"-"+pos_  in REDIportal_sites:
              continue  #remove all sites in REDIportal_sites from neg set

         true_value_all.append(0)
         index_neg.append(len(true_value_all)-1)

    predict_value_all.append(predictscore)
    num_noncandidate+=1


true_value_all=np.asarray(true_value_all)
predict_value_all=np.asarray(predict_value_all)
if balance_neg:
    np.random.seed(1234)
    np.random.shuffle(index_neg)
pos_coverage_ratio = 0.1
if balance_neg:
    index_pos=np.where(true_value_all > pos_coverage_ratio)[0]
    index_pos_len=len(index_pos)
    select_neg=index_neg[:index_pos_len]
    predict_value_selected=predict_value_all[np.concatenate([index_pos,select_neg])]
    true_value_selected=true_value_all[np.concatenate([index_pos,select_neg])]
else:
     predict_value_selected=predict_value_all
     true_value_selected=true_value_all

true_value_selected = (true_value_selected > pos_coverage_ratio).astype(int) 

HEK293T_true_value_selected = true_value_selected
HEK293T_predict_value_selected = predict_value_selected

In [5]:
covfilename=5
coverage_cutoff= 5
datatype = 'HCT116_WT'
balance_neg = True
inputfolder="/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure2e/ReDD_results/release_general_model"

filename1=inputfolder + '/'+"HCT116_WT_onlycandidate_cov"+str(covfilename)+"_ratio0_modcov0.sitelev.bed" #background must use no filter except coverage
filename2=inputfolder + '/'+"HCT116_WT_noncandidate_cov"+str(covfilename)+"_ratio0_modcov0.sitelev.bed"

candidatefile="/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure2a/reditools2_candidates/"+hash_candidat[datatype]
rediportal = "/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure3a/REDIportal_hg38.txt"

using_mean_prediction_score=True
if using_mean_prediction_score:
   score_index=-1
else:
   score_index=-2

true_candidate_ratios={}
shortreadcoverage={}
input = open(candidatefile,'r')
for line in input:
    chr_ = line.split()[0]
    pos_ = line.split()[1]
    true_candidate_ratios[chr_+"-"+pos_]=float(line.split("\t")[3])
    shortreadcoverage[chr_+"-"+pos_]=float(line.split("\t")[-1])

REDIportal_sites={}
input = open(rediportal,'r')
for line in input:
    chr_ = line.split()[0]
    pos_ = line.split()[1]
    REDIportal_sites[chr_+"-"+pos_]=1


true_value_all=[]
predict_value_all=[]
num_candidate=0
index_neg=[]
input=open(filename1)
for line in input:
    readcoverage=int(line.split()[3])
    if readcoverage<coverage_cutoff:
         continue

    chr_ = line.split()[0]
    pos_ = line.split()[1]
    if len(line.split("\t")) == 6:
        predictscore=float(line.split("\t")[score_index]) #last score is mean prediciton score
    else:
        predictscore=float(line.split("\t")[-1]) #for old result there is no mean prediction scores

    if chr_+"-"+pos_ in true_candidate_ratios:
         true_value_all.append(true_candidate_ratios[chr_+"-"+pos_])
    else:
         if chr_+"-"+pos_  in REDIportal_sites:
              continue  #remove all sites in REDIportal_sites from neg set

         true_value_all.append(0)
         index_neg.append(len(true_value_all)-1)

    predict_value_all.append(predictscore)
    num_candidate+=1

input=open(filename2)
num_noncandidate=0
for line in input:
    readcoverage=int(line.split()[3])
    if readcoverage<coverage_cutoff:
         continue

    chr_ = line.split()[0]
    pos_ = line.split()[1]
    if len(line.split("\t")) == 6:
        predictscore=float(line.split("\t")[score_index])
    else:
        predictscore=float(line.split("\t")[-1])

    if chr_+"-"+pos_ in true_candidate_ratios:
         true_value_all.append(true_candidate_ratios[chr_+"-"+pos_])
    else:
         if chr_+"-"+pos_  in REDIportal_sites:
              continue  #remove all sites in REDIportal_sites from neg set

         true_value_all.append(0)
         index_neg.append(len(true_value_all)-1)

    predict_value_all.append(predictscore)
    num_noncandidate+=1


true_value_all=np.asarray(true_value_all)
predict_value_all=np.asarray(predict_value_all)
if balance_neg:
    np.random.seed(1234)
    np.random.shuffle(index_neg)
pos_coverage_ratio = 0.1
if balance_neg:
    index_pos=np.where(true_value_all > pos_coverage_ratio)[0]
    index_pos_len=len(index_pos)
    select_neg=index_neg[:index_pos_len]
    predict_value_selected=predict_value_all[np.concatenate([index_pos,select_neg])]
    true_value_selected=true_value_all[np.concatenate([index_pos,select_neg])]
else:
     predict_value_selected=predict_value_all
     true_value_selected=true_value_all

true_value_selected = (true_value_selected > pos_coverage_ratio).astype(int) 

HCT116_true_value_selected = true_value_selected
HCT116_predict_value_selected = predict_value_selected

In [10]:
def get_curves(true_value_selected,predict_value_selected):
    # roc
    fpr, tpr,thres = roc_curve(true_value_selected, predict_value_selected)
    sel_fpr = fpr[np.abs(fpr - 0.05).argmin()]
    sel_tpr = tpr[np.abs(fpr - 0.05).argmin()]
    sel_roc_thres = thres[np.abs(fpr - 0.05).argmin()]
    roc_auc = auc(fpr, tpr)
    predictions = predict_value_selected >= sel_roc_thres
    correct_roc_thres = predictions == true_value_selected
    # pr
    precision, recall, thresholds = precision_recall_curve(true_value_selected, predict_value_selected)
    sel_precision = precision[np.abs(precision - 0.95).argmin()]
    sel_recall =  recall[np.abs(precision - 0.95).argmin()]
    self_pr_thres = thresholds[np.abs(precision - 0.95).argmin()]
    precision_roc_auc = average_precision_score(true_value_selected, predict_value_selected)
    
    predictions = predict_value_selected >= self_pr_thres
    correct_pr_thres = predictions == true_value_selected
    return fpr,tpr,thres,roc_auc,sel_fpr,sel_tpr,sel_roc_thres,precision,recall,thresholds,precision_roc_auc,sel_precision,sel_recall,self_pr_thres,correct_roc_thres,correct_pr_thres

In [11]:
# export of ROC and PR
import pandas as pd
from pathlib import Path
sel_rows = []
correct_sites = []
for met,predict_value_selected,true_value_selected in zip(['HEK293T_independent_sample','HCT116'],[HEK293T_predict_value_selected,HCT116_predict_value_selected],[HEK293T_true_value_selected,HCT116_true_value_selected]):
    fpr,tpr,roc_thres,roc_auc,sel_fpr,sel_tpr,sel_roc_thres,precision,recall,pr_thres,precision_roc_auc,sel_precision,sel_recall,self_pr_thres,\
    correct_roc_thres,correct_pr_thres = get_curves(true_value_selected,predict_value_selected)
    roc_df = pd.DataFrame([fpr,tpr,roc_thres]).T
    roc_df.columns = ['FPR','TPR','ROC_thres']
    pr_df = pd.DataFrame([precision,recall,pr_thres]).T
    pr_df.columns = ['Precision','Recall','PR_thres']
    Path(f'plot_data/ROC/').mkdir(exist_ok=True,parents=True)
    Path(f'plot_data/PR/').mkdir(exist_ok=True,parents=True)
    roc_df.to_csv(f'plot_data/ROC/{met}.tsv',sep='\t',index=False)
    pr_df.to_csv(f'plot_data/PR/{met}.tsv',sep='\t',index=False)
    sel_rows.append([met,roc_auc,precision_roc_auc,sel_fpr,sel_tpr,sel_roc_thres,sel_precision,sel_recall,self_pr_thres])
    correct_sites.append([met,correct_roc_thres,correct_pr_thres])
summary_df = pd.DataFrame(sel_rows,columns=['Method','ROC_AUC','PR_AUC','FPR','TPR','ROC_threshold','Precision','Recall','PR_threshold'])
summary_df.to_csv('plot_data/AUC_and_stats_when_controlling_FPR_and_precision.tsv',sep='\t',index=False)

In [12]:
def cal_contigency_table(rowA,rowB):
    roc_contigency = [[len(np.where((rowA['correct_ROC_threshold'] == True) & (rowB['correct_ROC_threshold']==True))[0]),
    len(np.where((rowA['correct_ROC_threshold'] == True) & (rowB['correct_ROC_threshold']==False))[0])],
    [len(np.where((rowA['correct_ROC_threshold'] == False) & (rowB['correct_ROC_threshold']==True))[0]),
    len(np.where((rowA['correct_ROC_threshold'] == False) & (rowB['correct_ROC_threshold']==False))[0])]]
    
    pr_contigency = [[len(np.where((rowA['correct_PR_threshold'] == True) & (rowB['correct_PR_threshold']==True))[0]),
    len(np.where((rowA['correct_PR_threshold'] == True) & (rowB['correct_PR_threshold']==False))[0])],
    [len(np.where((rowA['correct_PR_threshold'] == False) & (rowB['correct_PR_threshold']==True))[0]),
    len(np.where((rowA['correct_PR_threshold'] == False) & (rowB['correct_PR_threshold']==False))[0])]]
    return roc_contigency,pr_contigency

In [13]:
import statsmodels.stats.contingency_tables
correct_df = pd.DataFrame(correct_sites,columns=['Method','correct_ROC_threshold','correct_PR_threshold'])

In [15]:
roc_contigency,pr_contigency = cal_contigency_table(correct_df.iloc[0],correct_df.iloc[1])
print(statsmodels.stats.contingency_tables.mcnemar(roc_contigency, exact=True, correction=True).pvalue)
print(statsmodels.stats.contingency_tables.mcnemar(pr_contigency, exact=True, correction=True).pvalue)

ValueError: operands could not be broadcast together with shapes (5232,) (4646,) 

In [120]:
#export of IGV
example_df = example_df.join(pd.Series(Dinopore_ratio_all,name='Dinopore_ratio'),on='Site')
example_df['ReDD_prediction'] = example_df['ReDD_ratio'].apply(lambda x:'editing' if x>=redd_thres else 'non-editing')
example_df['Dinopore_prediction'] = example_df['Dinopore_prob'].apply(lambda x:'editing' if x>=dinopore_thres else 'non-editing')
example_df['DeepEdit_prediction'] = example_df['DeepEdit_ratio'].apply(lambda x:'editing' if x>=deepedit_thres else 'non-editing')
example_df['ReDD_threshold'] = redd_thres
example_df['Dinopore_threshold'] = dinopore_thres
example_df['DeepEdit_threshold'] = deepedit_thres

In [124]:
example_df.to_csv('plot_data/IGV.tsv',sep='\t',index=False)

In [123]:
example_df

,Site,truth_ratio,ReDD_ratio,Dinopore_prob,DeepEdit_ratio,Dinopore_ratio,ReDD_prediction,Dinopore_prediction,DeepEdit_prediction,ReDD_threshold,Dinopore_threshold,DeepEdit_threshold
40,chr3-136334873,0.29,0.134,0.071846,0.0,0.000000,editing,non-editing,non-editing,0.051,0.373703,0.459459
424,chr3-136334891,0.1,0.035,0.788267,0.0,0.249500,non-editing,editing,non-editing,0.051,0.373703,0.459459
425,chr3-136334901,0.44,0.139,0.962077,0.0,0.351838,editing,editing,non-editing,0.051,0.373703,0.459459
426,chr3-136334941,0.3,0.341,0.880085,0.0,0.421711,editing,editing,non-editing,0.051,0.373703,0.459459
427,chr3-136334944,0.19,0.014,0.664913,0.0,0.168159,non-editing,editing,non-editing,0.051,0.373703,0.459459
837,chr3-136335628,0.56,0.149,0.03255,0.117647,0.000000,editing,non-editing,non-editing,0.051,0.373703,0.459459
838,chr3-136335690,0.4,0.001,0.231032,0.1,0.000000,non-editing,non-editing,non-editing,0.051,0.373703,0.459459
839,chr3-136335701,0.19,0.133,0.082467,0.1,0.000000,editing,non-editing,non-editing,0.051,0.373703,0.459459
840,chr3-136335709,0.89,0.705,0.001343,0.6,0.000000,editing,non-editing,editing,0.051,0.373703,0.459459
